In [1]:
# 필요한 라이브러리 임포트
from numba import njit  # JIT 컴파일러 (Python 코드를 기계어로 컴파일하여 성능 향상)
import numpy as np  # 수치 계산용 라이브러리

# HFT 백테스트 프레임워크의 핵심 클래스들
from hftbacktest import BacktestAsset, HashMapMarketDepthBacktest

In [ ]:
# 주문 타입 및 상태 상수 임포트
# LIMIT: 지정가 주문, GTC: Good Till Cancel (취소될 때까지 유효)
# NONE: 상태 없음, NEW: 신규, FILLED: 체결완료, CANCELED: 취소됨, EXPIRED: 만료됨
from hftbacktest import LIMIT, GTC, NONE, NEW, FILLED, CANCELED, EXPIRED

@njit  # Numba JIT 컴파일 데코레이터 - 실행 속도 향상
def print_orders(hbt):
    """현재 모든 주문의 상태를 출력하는 함수"""
    
    # 자산 인덱스 0번의 주문 목록 가져오기
    orders = hbt.orders(0)

    # 주문 목록을 순회하기 위한 이터레이터
    order_values = orders.values()
    while order_values.has_next():
        order = order_values.get()

        # 주문 상태(status)를 문자열로 변환
        order_status = ''
        if order.status == NONE:
            order_status = 'NONE'  
        elif order.status == NEW:
            order_status = 'NEW'
        elif order.status == FILLED:
            order_status = 'FILLED'
        elif order.status == CANCELED:
            order_status = 'CANCELED'
        elif order.status == EXPIRED:
            order_status = 'EXPIRED'

        # 주문 요청(req) 상태를 문자열로 변환
        if order.req == NONE:
            order_req = 'NONE'
        elif order.req == NEW:
            order_req = 'NEW'
        elif order.req == CANCELED:
            order_req = 'CANCELED'

        # 주문 정보 출력
        print(
            'current_timestamp:', hbt.current_timestamp,  # 현재 시뮬레이션 시각
            ', order_id:', order.order_id,  # 주문 ID
            ', order_price:', np.round(order.price, 1),  # 주문 가격 (소수점 1자리)
            ', order_qty:', order.qty,  # 주문 수량
            ', order_status:', order_status,  # 주문 상태
            ', order_req:', order_req  # 주문 요청 상태
        )

@njit
def submit_order(hbt):
    """백테스트 시뮬레이션을 실행하고 매수 주문을 제출하는 메인 함수"""
    
    is_order_submitted = False  # 주문 제출 여부 플래그
    
    # 30초(30 * 1e9 나노초)씩 시간을 진행하며 시뮬레이션 실행
    # elapse()가 0을 반환하면 시뮬레이션 계속, 그 외는 종료
    while hbt.elapse(30 * 1e9) == 0:
        print_orders(hbt)  # 현재 주문 상태 출력

        # 자산 0번의 호가창(order book depth) 정보 가져오기
        depth = hbt.depth(0)

        # 아직 주문을 제출하지 않았다면 주문 제출
        if not is_order_submitted:
            
            order_id = 1  # 주문 고유 ID
            # 최우선 매수호가에서 300틱 아래 가격으로 주문 (유동성 공급자 역할)
            order_price = depth.best_bid - 300 * depth.tick_size
            order_qty = 1  # 주문 수량
            time_in_force = GTC  # 주문 유효 기간: 취소될 때까지
            order_type = LIMIT  # 주문 타입: 지정가
            
            # 매수 주문 제출
            # 인자: (자산 인덱스, 주문ID, 가격, 수량, 유효기간, 주문타입, 사후거래여부)
            hbt.submit_buy_order(0, order_id, order_price, order_qty, time_in_force, order_type, False)
            is_order_submitted = True  # 플래그 업데이트
            
    return True

In [3]:
# 백테스트 자산(asset) 설정
asset = (
    BacktestAsset()
    .data(['BTCUSDT_20240626.npz'])  # 시장 데이터 파일 (BTCUSDT 2024년 6월 26일)
    .initial_snapshot('BTCUSDT_20240626.npz')  # 초기 호가창 스냅샷
    .linear_asset(1.0)  # 선형 자산 (선물/스왑 계약), 1배 레버리지
    .constant_order_latency(10_000_000, 10_000_000)  # 주문 지연시간 (나노초): (거래소→로컬, 로컬→거래소) = 10ms씩
    .no_partial_fill_exchange()  # 거래소에서 부분 체결 없음 (전량 체결 or 미체결)
    .trading_value_fee_model(0.0002, 0.0007)  # 수수료 모델: (maker 0.02%, taker 0.07%)
    .tick_size(0.1)  # 최소 가격 단위 (틱 사이즈): 0.1 USDT
    .lot_size(0.001)  # 최소 주문 수량 단위: 0.001 BTC
    .last_trades_capacity(0)  # 최근 거래 내역 저장 용량 (0 = 저장 안함)
)

# HashMap 기반 호가창 백테스터 생성
hbt = HashMapMarketDepthBacktest([asset])

# 주문 제출 및 시뮬레이션 실행
submit_order(hbt)

# 백테스트 종료 및 결과 정리
_ = hbt.close()

current_timestamp: 1719360060007000000 , order_id: 1 , order_price: 61757.6 , order_qty: 1.0 , order_status: NEW , order_req: NONE
current_timestamp: 1719360090007000000 , order_id: 1 , order_price: 61757.6 , order_qty: 1.0 , order_status: NEW , order_req: NONE
current_timestamp: 1719360120007000000 , order_id: 1 , order_price: 61757.6 , order_qty: 1.0 , order_status: NEW , order_req: NONE
current_timestamp: 1719360150007000000 , order_id: 1 , order_price: 61757.6 , order_qty: 1.0 , order_status: FILLED , order_req: NONE
current_timestamp: 1719360180007000000 , order_id: 1 , order_price: 61757.6 , order_qty: 1.0 , order_status: FILLED , order_req: NONE
current_timestamp: 1719360210007000000 , order_id: 1 , order_price: 61757.6 , order_qty: 1.0 , order_status: FILLED , order_req: NONE
current_timestamp: 1719360240007000000 , order_id: 1 , order_price: 61757.6 , order_qty: 1.0 , order_status: FILLED , order_req: NONE
current_timestamp: 1719360270007000000 , order_id: 1 , order_price: 617